In [0]:
%pip install s3fs
%pip install sentence-transformers

%pip install s3fs
%pip install wordcloud
%pip install gensim
%pip install --upgrade --force-reinstall numpy==1.22.0

Python interpreter will be restarted.
  Using cached s3fs-2023.3.0-py3-none-any.whl (27 kB)
  Using cached aiohttp-3.8.4-cp39-cp39-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (1.0 MB)
  Using cached fsspec-2023.3.0-py3-none-any.whl (145 kB)
  Using cached aiobotocore-2.4.2-py3-none-any.whl (66 kB)
  Using cached aioitertools-0.11.0-py3-none-any.whl (23 kB)
  Using cached botocore-1.27.59-py3-none-any.whl (9.1 MB)
  Using cached frozenlist-1.3.3-cp39-cp39-manylinux_2_5_x86_64.manylinux1_x86_64.manylinux_2_17_x86_64.manylinux2014_x86_64.whl (158 kB)
  Using cached yarl-1.8.2-cp39-cp39-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (264 kB)
  Using cached async_timeout-4.0.2-py3-none-any.whl (5.8 kB)
  Using cached aiosignal-1.3.1-py3-none-any.whl (7.6 kB)
  Using cached multidict-6.0.4-cp39-cp39-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (114 kB)
  Attempting uninstall: botocore
    Found existing installation: botocore 1.24.32
    Not uninstalling botocore at /databricks/python3/

# Load Data

In [0]:
import pandas as pd
df = pd.read_csv(".../control_fg_tp_km.csv", header=0)

## Preprocessing

In [0]:
import re
def standardize_redaction(text):
    res_text = re.sub('XXX+', '[The organization]', text)
    res_text = re.sub('AAA+', '[The organization]', res_text)
    res_text = re.sub('\[The organization\]', '[The organization]'.upper(), res_text)
    return res_text

print(df[df['control'].apply(lambda x: "XXXXXXXX" in x)]['control'].count())
print(df[df['control'].apply(lambda x: "AAAAAAAA" in x)]['control'].count())

df['control_processed'] = df['control'].apply(standardize_redaction)

# Remove punctuation
df['control_processed'] = df['control_processed'].map(lambda x: re.sub('[,\.!?]', '', x))

0
0


In [0]:
df_framework_section_stats = df.groupby("framework_section_code").agg({"control": "count"}).reset_index()
df_framework_section_stats

,framework_section_code,control
0,A1.1,401
1,A1.2,1128
2,A1.3,392
3,C1.1,322
4,C1.2,212
...,...,...
65,PI1.1,112
66,PI1.2,122
67,PI1.3,175
68,PI1.4,122


In [0]:
target_framework_section_code = df_framework_section_stats[df_framework_section_stats["control"] >= 10]["framework_section_code"].tolist()

In [0]:
df_selected = df[df["framework_section_code"].isin(target_framework_section_code)]

## Split the data

In [0]:
from sklearn.model_selection import train_test_split

In [0]:
df_train, df_test = train_test_split(df_selected, test_size=0.20, stratify=df_selected.framework_section_code)

In [0]:
df_train.shape, df_test.shape

Out[8]: ((39542, 29), (9886, 29))

In [0]:
df_train.to_csv(".../df_train.csv", header=True, index=False)
df_test.to_csv(".../df_test.csv", header=True, index=False)

# Baselines

In [0]:
# TF-IDF Feature Generation
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import CountVectorizer
from nltk.tokenize import RegexpTokenizer

# TF-IDF Feature Generation
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import CountVectorizer
from nltk.tokenize import RegexpTokenizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.svm import SVC
from sklearn.metrics import classification_report
from sklearn.linear_model import LogisticRegression
from lightgbm import LGBMClassifier
from sklearn.pipeline import Pipeline
import matplotlib.pyplot as plt
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay

In [0]:
def train(model_pipeline, X_data, X_true):
    return model_pipeline.fit(X_data, X_true)

def evaluate(model_pipeline, y_data, y_true, cm=False):
    y_pred = model_pipeline.predict(y_data)
    
    print(classification_report(y_true, y_pred))
    
    if cm:
        # confusion metric
        cm = confusion_matrix(y_true, y_pred)    
        cmp = ConfusionMatrixDisplay(confusion_matrix=cm, display_labels=model_pipeline["model"].classes_)

        plt.figure(figsize=(10,10))
        fig, ax = plt.subplots(figsize=(10,10))
        cmp.plot(ax=ax, xticks_rotation="vertical", cmap="cool")

## Logreg + CountVectorizer

In [0]:
pipe_logreg = Pipeline([('vectorizer', CountVectorizer(ngram_range=(1, 3), stop_words="english")), ('model', LogisticRegression(C=1e5))])
pipe_logreg = train(pipe_logreg, df_train.control_processed, df_train.framework_section_code)
evaluate(pipe_logreg, df_test.control_processed, df_test.framework_section_code)

/databricks/python/lib/python3.9/site-packages/sklearn/linear_model/_logistic.py:814: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
              precision    recall  f1-score   support

        A1.1       0.58      0.35      0.44        80
        A1.2       0.46      0.38      0.42       226
        A1.3       0.49      0.28      0.36        78
        C1.1       0.33      0.41      0.37        64
        C1.2       0.39      0.36      0.38        42
       CC1.0       0.00      0.00      0.00         6
       CC1.1       0.45      0.37      0.41       292
       CC1.2       0.46      0.46      0.46      

## MultinomialNB + CountVectorizer

In [0]:
pipe_nb = Pipeline([('vectorizer', CountVectorizer(ngram_range=(1, 3), stop_words="english")), ('model', MultinomialNB())])
pipe_nb = train(pipe_nb, df_train.control_processed, df_train.framework_section_code)
evaluate(pipe_nb, df_test.control_processed, df_test.framework_section_code)

/databricks/python/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
              precision    recall  f1-score   support

        A1.1       0.77      0.30      0.43        80
        A1.2       0.57      0.36      0.44       226
        A1.3       0.49      0.31      0.38        78
        C1.1       0.61      0.31      0.41        64
        C1.2       0.52      0.26      0.35        42
       CC1.0       0.00      0.00      0.00         6
       CC1.1       0.52      0.41      0.46       292
       CC1.2       0.43      0.70      0.53       165
       CC1.3       0.49      0.61      0.55       223
       CC1.4       0.50      0.53      0.51       294
       CC1.5       0.42      0.38      0.40       229
       CC2.0       0.00      0.00      0.00  

## Logreg + Bert

In [0]:
from sentence_transformers import SentenceTransformer

embedder = SentenceTransformer('distilbert-base-nli-stsb-mean-tokens', device=0)

def train_bert(model, X_data, X_true):
    return model.fit(embedder.encode(X_data.tolist()), X_true)

def evaluate_bert(model, y_data, y_true, cm=False):
    y_pred = model.predict(embedder.encode(y_data.tolist()))
    
    print(classification_report(y_true, y_pred))
    
    if cm:
        # confusion metric
        cm = confusion_matrix(y_true, y_pred)    
        cmp = ConfusionMatrixDisplay(confusion_matrix=cm, display_labels=model.classes_)

        plt.figure(figsize=(10,10))
        fig, ax = plt.subplots(figsize=(10,10))
        cmp.plot(ax=ax, xticks_rotation="vertical", cmap="cool")

In [0]:
logreg_bert = train_bert(LogisticRegression(C=1e5), df_train.control_processed, df_train.framework_section_code)
evaluate_bert(logreg_bert, df_test.control_processed, df_test.framework_section_code)

/databricks/python/lib/python3.9/site-packages/sklearn/linear_model/_logistic.py:814: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
              precision    recall  f1-score   support

        A1.1       0.45      0.62      0.52        80
        A1.2       0.36      0.34      0.35       226
        A1.3       0.51      0.31      0.38        78
        C1.1       0.41      0.28      0.33        64
        C1.2       0.41      0.43      0.42        42
       CC1.0       0.00      0.00      0.00         6
       CC1.1       0.41      0.48      0.44       292
       CC1.2       0.44      0.58      0.50      

# Add Topic Modeling and KMeans Reuslt as Feature

## Topic Modeling

In [0]:
import gensim
from gensim.utils import simple_preprocess
import nltk
import gensim.corpora as corpora
from gensim import models
from pprint import pprint
nltk.download('stopwords')
from nltk.corpus import stopwords
import re
from gensim.models.coherencemodel import CoherenceModel
from matplotlib import pyplot as plt

stop_words = stopwords.words('english')

def sent_to_words(sentences):
    for sentence in sentences:
        # deacc=True removes punctuations
        yield(gensim.utils.simple_preprocess(str(sentence), deacc=True))
        
def remove_stopwords(texts):
    return [[word for word in simple_preprocess(str(doc)) if word not in stop_words] for doc in texts]

def build_corpus(df):
    # Remove punctuation
    df['control_processed'] = df['control_processed'].map(lambda x: re.sub('[,\.!?]', '', x))

    # Convert the titles to lowercase
    df['control_processed'] = df['control_processed'].map(lambda x: x.lower())

    # Tokenization
    texts = df["control_processed"].tolist()
    texts_words = list(sent_to_words(texts))

    # Remove stop words
    texts_words = remove_stopwords(texts_words)
    
    # Create Dictionary
    id2word = corpora.Dictionary(texts_words)

    # Remove tokens appear less than 10 times and in more than 90% of the docs
    id2word.filter_extremes(no_below=10, no_above=0.5)

    # Create Corpus
    corpus = [id2word.doc2bow(text) for text in texts_words]
    # print(corpus[:1][0][:30])

    return texts_words, id2word, corpus

def tuning_best_LDA(max_num_topics, texts_words, corpus, id2word):
    candidate_models = []
    MAX_NUM_TOPICS = 15
    for n in range(2, max_num_topics + 1):
        lda_model = gensim.models.LdaMulticore(corpus=corpus, id2word=id2word, num_topics=n, passes=5, iterations=100)

        # Topic Coherence measures score a single topic by measuring the degree of semantic similarity between high scoring words in the topic. 
        # These measurements help distinguish between topics that are semantically interpretable topics 
        # and topics that are artifacts of statistical inference.
        coherence_model_lda = CoherenceModel(model=lda_model, texts=texts_words, dictionary=id2word, coherence='c_v')
        coherence = coherence_model_lda.get_coherence()
        candidate_models.append((n, lda_model, coherence))
        print("num topic: ", n, 'Coherence Score: ', coherence)
    return candidate_models

def plot_coherence_score(num_topics, scores):
    plt.plot(range(2, num_topics+1), scores)
    plt.xlabel('Num Topics')
    plt.ylabel('Coherence Score')
    # plt.savefig('elbow.png')
    plt.show()
    
def get_topic_distribution(model, dictionary, text):
    text_processed = remove_stopwords(sent_to_words([text]))[0]
    
    corpus = dictionary.doc2bow(text_processed)
    topics = model[corpus]

    # sort topics by prop
    topics = sorted(topics, key=lambda x: (x[1]), reverse=True)
    result_dict = []
    for i, (topic_id, prop_topic) in enumerate(topics):
        wp = model.show_topic(topic_id)
        
        topic_keywords = "_".join([word for word, prop in wp])
        result_dict.append({"topic_index":topic_id, "topic_prob": round(prop_topic,4), "topic_keywords": topic_keywords})
    return result_dict

def assign_topic_prediction(model, df, id2word):
    df["gensim_topics"] = df['control_processed'].apply(lambda x: get_topic_distribution(model, id2word, x))
    df["gensim_topic_tp1"] = df["gensim_topics"].apply(lambda x: x[0])
    df["gensim_tp1_topic_id"] = df["gensim_topic_tp1"].apply(lambda x: x['topic_index'])
    df["gensim_tp1_topic_prob"] = df["gensim_topic_tp1"].apply(lambda x: x['topic_prob'])
    df["gensim_tp1_topic_keywords"] = df["gensim_topic_tp1"].apply(lambda x: x['topic_keywords'])
    
    return df

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [0]:
train_texts_words, train_id2word, train_corpus = build_corpus(df_train)

In [0]:
num_topics = 8

lda_model = gensim.models.LdaMulticore(corpus=train_corpus, id2word=train_id2word, num_topics=num_topics, passes=5, iterations=100)

coherence_model_lda = CoherenceModel(model=lda_model, texts=train_texts_words, dictionary=train_id2word, coherence='c_v')
coherence = coherence_model_lda.get_coherence()
print("num topic: ", num_topics, 'Coherence Score: ', coherence)

num topic:  8 Coherence Score:  0.4282632232628161


In [0]:
import pyLDAvis.gensim
import pickle 
import pyLDAvis
import os
import s3fs

LDAvis_prepared = pyLDAvis.gensim.prepare(lda_model, train_corpus, train_id2word)

pyLDAvis.display(LDAvis_prepared)

/local_disk0/.ephemeral_nfs/cluster_libraries/python/lib/python3.9/site-packages/pyLDAvis/_prepare.py:243: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only.
  default_term_info = default_term_info.sort_values(


In [0]:
def get_topic_distribution(model, dictionary, text):
    text_processed = remove_stopwords(sent_to_words([text]))[0]
    
    corpus = dictionary.doc2bow(text_processed)
    topics = model[corpus]

    # sort topics by prop
    topics = sorted(topics, key=lambda x: (x[1]), reverse=True)
    result_dict = []
    for i, (topic_id, prop_topic) in enumerate(topics):
        wp = model.show_topic(topic_id)
        
        topic_keywords = "_".join([word for word, prop in wp])
        result_dict.append({"topic_index":topic_id, "topic_prob": round(prop_topic,4), "topic_keywords": topic_keywords})
    return result_dict

def assign_topic_prediction(model, df, id2word):
    df["gensim_topics"] = df['control_processed'].apply(lambda x: get_topic_distribution(model, id2word, x))
    df["gensim_topic_tp1"] = df["gensim_topics"].apply(lambda x: x[0])
    df["gensim_tp1_topic_id"] = df["gensim_topic_tp1"].apply(lambda x: x['topic_index'])
    df["gensim_tp1_topic_prob"] = df["gensim_topic_tp1"].apply(lambda x: x['topic_prob'])
    df["gensim_tp1_topic_keywords"] = df["gensim_topic_tp1"].apply(lambda x: x['topic_keywords'])
    
    return df

In [0]:
df_train = df_train[['org_id', 'control', 'framework_section_code', 'framework_section_title', 'source', 'control_category', 'control_sub_category', 'control_shortname', 'control_processed']]
df_train = assign_topic_prediction(lda_model, df_train, train_id2word) 

# use the prob of each topic as vec.
for i in range(lda_model.num_topics):
    df_train[f"gensim_tp{i}_prob"] = df_train["gensim_topics"].apply(lambda x: max([t["topic_prob"] for t in x if t["topic_index"] == i] + [0]))

In [0]:
df_test = df_test[['org_id', 'control', 'framework_section_code', 'framework_section_title', 'source', 'control_category', 'control_sub_category', 'control_shortname', 'control_processed']]
df_test = assign_topic_prediction(lda_model, df_test, train_id2word) 
# use the prob of each topic as vec.
for i in range(lda_model.num_topics):
    df_test[f"gensim_tp{i}_prob"] = df_test["gensim_topics"].apply(lambda x: max([t["topic_prob"] for t in x if t["topic_index"] == i] + [0]))

In [0]:
df_train.to_csv(".../df_train_tp.csv", header=True, index=False)
df_test.to_csv(".../df_test_tp.csv", header=True, index=False)

In [0]:
df_train = pd.read_csv(".../df_train_tp.csv", header=0)
df_test = pd.read_csv(".../df_test_tp.csv", header=0)

## Kmeans

In [0]:
# TF-IDF Feature Generation
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import CountVectorizer
from nltk.tokenize import RegexpTokenizer

from sklearn.cluster import KMeans

# Initialize regex tokenizer
tokenizer = RegexpTokenizer(r'\w+')

# Vectorize document using TF-IDF
tf_idf_vect = TfidfVectorizer(lowercase=True,
                        stop_words='english',
                        ngram_range = (1,1),
                        tokenizer = tokenizer.tokenize)
# Fit and Transfrom Text Data
tfidf_corpus = tf_idf_vect.fit_transform(df_train['control_processed'])

# Check Shape of Count Vector
tfidf_corpus.shape, tfidf_corpus

Out[53]: ((39542, 6809),
 <39542x6809 sparse matrix of type '<class 'numpy.float64'>'
 	with 550887 stored elements in Compressed Sparse Row format>)

In [0]:
kmeans = KMeans(n_clusters=12, random_state=42)
kmeans.fit(tfidf_corpus)

Exception ignored on calling ctypes callback function: <function _ThreadpoolInfo._find_modules_with_dl_iterate_phdr.<locals>.match_module_callback at 0x7fd9150e9af0>
Traceback (most recent call last):
  File "/databricks/python/lib/python3.9/site-packages/threadpoolctl.py", line 400, in match_module_callback
    self._make_module_from_path(filepath)
  File "/databricks/python/lib/python3.9/site-packages/threadpoolctl.py", line 515, in _make_module_from_path
    module = module_class(filepath, prefix, user_api, internal_api)
  File "/databricks/python/lib/python3.9/site-packages/threadpoolctl.py", line 606, in __init__
    self.version = self.get_version()
  File "/databricks/python/lib/python3.9/site-packages/threadpoolctl.py", line 646, in get_version
    config = get_config().split()
AttributeError: 'NoneType' object has no attribute 'split'
Out[54]: KMeans(n_clusters=12, random_state=42)

In [0]:
df_train["kmeans_cluster_id_tfidf"] = kmeans.labels_
df_test["kmeans_cluster_id_tfidf"] = kmeans.predict(tf_idf_vect.transform(df_test['control_processed']))

Exception ignored on calling ctypes callback function: <function _ThreadpoolInfo._find_modules_with_dl_iterate_phdr.<locals>.match_module_callback at 0x7fd914ebd430>
Traceback (most recent call last):
  File "/databricks/python/lib/python3.9/site-packages/threadpoolctl.py", line 400, in match_module_callback
    self._make_module_from_path(filepath)
  File "/databricks/python/lib/python3.9/site-packages/threadpoolctl.py", line 515, in _make_module_from_path
    module = module_class(filepath, prefix, user_api, internal_api)
  File "/databricks/python/lib/python3.9/site-packages/threadpoolctl.py", line 606, in __init__
    self.version = self.get_version()
  File "/databricks/python/lib/python3.9/site-packages/threadpoolctl.py", line 646, in get_version
    config = get_config().split()
AttributeError: 'NoneType' object has no attribute 'split'


In [0]:
df_train.to_csv(".../df_train_tp.csv", header=True, index=False)
df_test.to_csv(".../df_test_tp.csv", header=True, index=False)

# Add Topic Modeling/Kmeans as Feature

In [0]:
from sklearn.preprocessing import OneHotEncoder
from scipy.sparse import hstack

vectorizer = CountVectorizer(ngram_range=(1, 3), stop_words="english")
enc = OneHotEncoder()

def train_tp(model, X_data_text, X_data_num, X_true):
    X_data_text_vec = vectorizer.fit_transform(X_data_text)

    onehot_feat = enc.fit_transform(X_data_num)
    
    combined_features = hstack([X_data_text_vec, onehot_feat])

    return model.fit(combined_features, X_true)


def evaluate_tp(model, y_data_text, y_data_num, y_true, cm=False):
    y_data_text_vec = vectorizer.transform(y_data_text)
    
    onehot_feat = enc.transform(y_data_num)
    combined_features = hstack([y_data_text_vec, onehot_feat])
    
    y_pred = model.predict(combined_features)
    
    print(classification_report(y_true, y_pred))
    
    if cm:
        # confusion metric
        cm = confusion_matrix(y_true, y_pred)    
        cmp = ConfusionMatrixDisplay(confusion_matrix=cm, display_labels=model.classes_)

        plt.figure(figsize=(10,10))
        fig, ax = plt.subplots(figsize=(10,10))
        cmp.plot(ax=ax, xticks_rotation="vertical", cmap="cool")

## Topic: Logreg + CountVectorizer + Onehot

In [0]:
logreg_cnt_tp_oh = train_tp(LogisticRegression(C=1e5), df_train.control_processed, df_train[["gensim_tp1_topic_id"]], df_train.framework_section_code)
evaluate_tp(logreg_cnt_tp_oh, df_test.control_processed, df_test[["gensim_tp1_topic_id"]], df_test.framework_section_code, cm=False)

/databricks/python/lib/python3.9/site-packages/sklearn/linear_model/_logistic.py:814: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
              precision    recall  f1-score   support

        A1.1       0.55      0.46      0.50        80
        A1.2       0.46      0.34      0.39       226
        A1.3       0.48      0.29      0.37        78
        C1.1       0.34      0.41      0.37        64
        C1.2       0.40      0.38      0.39        42
       CC1.0       0.00      0.00      0.00         6
       CC1.1       0.42      0.38      0.39       292
       CC1.2       0.40      0.47      0.43      

## Kmeans: Logreg + CountVectorizer + Onehot

In [0]:
logreg_cnt_tp_km = train_tp(LogisticRegression(C=1e5), df_train.control_processed, df_train[["kmeans_cluster_id_tfidf"]], df_train.framework_section_code)
evaluate_tp(logreg_cnt_tp_km, df_test.control_processed, df_test[["kmeans_cluster_id_tfidf"]], df_test.framework_section_code, cm=False)

/databricks/python/lib/python3.9/site-packages/sklearn/linear_model/_logistic.py:814: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
              precision    recall  f1-score   support

        A1.1       0.56      0.49      0.52        80
        A1.2       0.39      0.33      0.36       226
        A1.3       0.48      0.28      0.35        78
        C1.1       0.31      0.41      0.35        64
        C1.2       0.41      0.38      0.40        42
       CC1.0       0.00      0.00      0.00         6
       CC1.1       0.43      0.38      0.41       292
       CC1.2       0.42      0.48      0.45      

## Topic: MultinomialNB + CountVectorizer + Onehot

In [0]:
nb_cnt_tp_oh = train_tp(MultinomialNB(), df_train.control_processed, df_train[["gensim_tp1_topic_id"]], df_train.framework_section_code)
evaluate_tp(nb_cnt_tp_oh, df_test.control_processed, df_test[["gensim_tp1_topic_id"]], df_test.framework_section_code, cm=False)

/databricks/python/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
              precision    recall  f1-score   support

        A1.1       0.77      0.30      0.43        80
        A1.2       0.56      0.35      0.43       226
        A1.3       0.48      0.29      0.37        78
        C1.1       0.62      0.31      0.42        64
        C1.2       0.47      0.21      0.30        42
       CC1.0       0.00      0.00      0.00         6
       CC1.1       0.52      0.42      0.46       292
       CC1.2       0.43      0.70      0.53       165
       CC1.3       0.40      0.68      0.51       223
       CC1.4       0.50      0.54      0.52       294
       CC1.5       0.42      0.36      0.39       229
       CC2.0       0.00      0.00      0.00  

## KMeans: MultinomialNB + CountVectorizer + Onehot

In [0]:
nb_cnt_tp_km = train_tp(MultinomialNB(), df_train.control_processed, df_train[["kmeans_cluster_id_tfidf"]], df_train.framework_section_code)
evaluate_tp(nb_cnt_tp_km, df_test.control_processed, df_test[["kmeans_cluster_id_tfidf"]], df_test.framework_section_code, cm=False)

/databricks/python/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
              precision    recall  f1-score   support

        A1.1       0.80      0.30      0.44        80
        A1.2       0.57      0.36      0.44       226
        A1.3       0.49      0.29      0.37        78
        C1.1       0.62      0.31      0.42        64
        C1.2       0.47      0.21      0.30        42
       CC1.0       0.00      0.00      0.00         6
       CC1.1       0.53      0.42      0.47       292
       CC1.2       0.42      0.69      0.53       165
       CC1.3       0.50      0.61      0.55       223
       CC1.4       0.50      0.53      0.52       294
       CC1.5       0.43      0.37      0.40       229
       CC2.0       0.00      0.00      0.00  

## Topic: Logreg + Bert + Onehot

In [0]:
from sentence_transformers import SentenceTransformer
import numpy as np

embedder = SentenceTransformer('distilbert-base-nli-stsb-mean-tokens', device=0)
enc = OneHotEncoder()

def train_bert_onehot(model, X_data_text, X_data_num, X_true):
    bert_vec = embedder.encode(X_data_text.tolist())
    
    enc.fit(X_data_num)
    onehot_feat = enc.transform(X_data_num).toarray()
    
    combined_features = np.hstack([bert_vec, onehot_feat])
    return model.fit(combined_features, X_true)

def evaluate_bert_onehot(model, y_data_text, y_data_num, y_true, cm=False):
    bert_vec = embedder.encode(y_data_text.tolist())
    
    onehot_feat = enc.transform(y_data_num).toarray()
    combined_features = np.hstack([bert_vec, onehot_feat])
    
    y_pred = model.predict(combined_features)
    
    print(classification_report(y_true, y_pred))
    
    if cm:
        # confusion metric
        cm = confusion_matrix(y_true, y_pred)    
        cmp = ConfusionMatrixDisplay(confusion_matrix=cm, display_labels=model.classes_)

        plt.figure(figsize=(10,10))
        fig, ax = plt.subplots(figsize=(10,10))
        cmp.plot(ax=ax, xticks_rotation="vertical", cmap="cool")
        


In [0]:
logreg_bert_tp_oh = train_bert_onehot(LogisticRegression(C=1e5), df_train.control_processed, df_train[["gensim_tp1_topic_id"]], df_train.framework_section_code)
evaluate_bert_onehot(logreg_bert_tp_oh, df_test.control_processed, df_test[["gensim_tp1_topic_id"]], df_test.framework_section_code, cm=False)

/databricks/python/lib/python3.9/site-packages/sklearn/linear_model/_logistic.py:814: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
              precision    recall  f1-score   support

        A1.1       0.54      0.34      0.42        80
        A1.2       0.36      0.43      0.39       226
        A1.3       0.42      0.37      0.39        78
        C1.1       0.41      0.33      0.37        64
        C1.2       0.39      0.40      0.40        42
       CC1.0       0.25      0.17      0.20         6
       CC1.1       0.43      0.46      0.45       292
       CC1.2       0.41      0.59      0.48      

## Kmeans: Logreg + Bert + Onehot

In [0]:
logreg_bert_tp_km = train_bert_onehot(LogisticRegression(C=1e5), df_train.control_processed, df_train[["kmeans_cluster_id_tfidf"]], df_train.framework_section_code)
evaluate_bert_onehot(logreg_bert_tp_km, df_test.control_processed, df_test[["kmeans_cluster_id_tfidf"]], df_test.framework_section_code, cm=False)

/databricks/python/lib/python3.9/site-packages/sklearn/linear_model/_logistic.py:814: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
              precision    recall  f1-score   support

        A1.1       0.64      0.35      0.45        80
        A1.2       0.44      0.35      0.39       226
        A1.3       0.28      0.64      0.39        78
        C1.1       0.42      0.34      0.38        64
        C1.2       0.41      0.50      0.45        42
       CC1.0       0.25      0.17      0.20         6
       CC1.1       0.43      0.45      0.44       292
       CC1.2       0.41      0.59      0.49      

In [0]:
# embedder = SentenceTransformer('distilbert-base-nli-stsb-mean-tokens', device=0)
# numerical_features_column = ["gensim_tp0_prob",	"gensim_tp1_prob",	"gensim_tp2_prob",	"gensim_tp3_prob",	"gensim_tp4_prob",	"gensim_tp5_prob",	"gensim_tp6_prob",	"gensim_tp7_prob"]

# def train_bert(model, X_data_text, X_data_num, X_true):
#     bert_vec = embedder.encode(X_data_text.tolist())
    
#     numerical_features = X_data_num.to_numpy()
#     combined_features = np.hstack([bert_vec, numerical_features])
#     return model.fit(combined_features, X_true)

# def evaluate_bert(model, y_data_text, y_data_num, y_true, cm=False):
#     bert_vec = embedder.encode(y_data_text.tolist())
#     numerical_features = y_data_num.to_numpy()
#     combined_features = np.hstack([bert_vec, numerical_features])
    
#     y_pred = model.predict(combined_features)
    
#     print(classification_report(y_true, y_pred))
    
#     if cm:
#         # confusion metric
#         cm = confusion_matrix(y_true, y_pred)    
#         cmp = ConfusionMatrixDisplay(confusion_matrix=cm, display_labels=model.classes_)

#         plt.figure(figsize=(10,10))
#         fig, ax = plt.subplots(figsize=(10,10))
#         cmp.plot(ax=ax, xticks_rotation="vertical", cmap="cool")

# logreg_bert_tp = train_bert(LogisticRegression(C=1e5), df_train.control_processed, df_train[numerical_features_column], df_train.framework_section_code)
# evaluate_bert(logreg_bert_tp, df_test.control_processed, df_test[numerical_features_column], df_test.framework_section_code, cm=False)